In [ ]:
import sys
from pathlib import Path

# Ubica la raíz del proyecto; desde tests/ es el padre del cwd
PROJECT_ROOT = Path.cwd().parent

import cv2 as cv
import numpy as np

run = 9
dataset = 4

# --- Configuración de paths ---
input_dir_base = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"
output_dir_base = PROJECT_ROOT / "Database" / "tmp" / f"imgPreprocTry{run:02d}"
output_dir_base.mkdir(parents=True, exist_ok=True)

labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

sigmas = [1.0, 2.0, 3.0, 4.0, 5.0]

Imgcfg = ImgPreprocCfg(
    target_size=512,
    sigma = 10.0,
    flag_refine_mask = False,
    open_ksize = 5,
    close_ksize = 5
)
pre = ImgPreproc(Imgcfg)

for l in labels:
    input_dir = input_dir_base / l
    output_dir = output_dir_base / l
    output_dir.mkdir(parents=True, exist_ok=True)
    
    for img_path in sorted(input_dir.glob("*.jpg")):
        img = cv.imread(str(img_path))
        if img is None:
            continue
        
        for sigmaX in sigmas:
            # --- 1) Pasar a HSV ---
            hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

            # --- 2) Rango de VERDE (OpenCV usa H en [0,179]) ---
            lower_green = np.array([35, 40, 40], dtype=np.uint8)
            upper_green = np.array([85, 255, 255], dtype=np.uint8)

            blurred = cv.GaussianBlur(hsv, (0, 0), sigmaX=sigmaX)

            out_name = f"{img_path.stem}_sigma{sigmaX:.1f}.png"
            out_path = output_dir / out_name
            cv.imwrite(str(out_path), blurred)

            # --- 3) Máscara de FONDO verde (255 = verde, 0 = no verde) ---
            mask_bg = cv.inRange(blurred, lower_green, upper_green)

            # --- 4) Máscara de OBJETO (invertimos) ---
            mask_obj = cv.bitwise_not(mask_bg)

            # (opcional) guardar para ver qué tal:
            cv.imwrite(str(output_dir / f"{img_path.stem}_{sigmaX}_mask_bg.png"), mask_bg)
            cv.imwrite(str(output_dir / f"{img_path.stem}_{sigmaX}_mask_obj.png"), mask_obj)



In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent  # si el cwd es tests/, el padre es la raíz del repo
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from Code.image import ImgPreproc


import cv2 as cv
import numpy as np


run = 1
dataset = 3

# --- Configuración de paths ---
input_dir = PROJECT_ROOT / "Database" / "data" / f"image{dataset}" / "Tornillo"
output_dir = PROJECT_ROOT / "Database" / "tmp" / f"imgPreprocTry{run:02d}"
output_dir.mkdir(parents=True, exist_ok=True)

labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]
print(f"Guardado: {out_path}")
sigmas = [1.0, 2.0, 3.0, 4.0, 5.0]

pre = ImgPreproc()

for img_path in sorted(input_dir.glob("*.jpg")):
    img = cv.imread(str(img_path))
    if img is None:
        continue
    
    img_mask = pre._normalize(img)

    out_name = f"{img_path.stem}_mask.png"
    out_path = output_dir / out_name
    cv.imwrite(str(out_path), img_mask)

    img_sq, mask_sq = pre._crop_and_square(img, img_mask)

    out_name_img_sq = f"{img_path.stem}_img_sq.png"
    out_name_mask_sq = f"{img_path.stem}_mask_sq.png"

    out_img_sq_path = output_dir / out_name_img_sq
    cv.imwrite(str(out_img_sq_path), img_sq)

    out_mask_sq_path = output_dir / out_name_mask_sq
    cv.imwrite(str(out_mask_sq_path), mask_sq)




In [3]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent  # si el cwd es tests/, el padre es la raíz del repo
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from Code.image import ImgPreproc, ImgPreprocCfg


import cv2 as cv
import numpy as np


run = 8
dataset = 4

# --- Configuración de paths ---
input_dir_base = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"
output_dir = PROJECT_ROOT / "Database" / "tmp" / f"imgPreprocTry{run:02d}"
output_dir.mkdir(parents=True, exist_ok=True)

labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]


Imgcfg = ImgPreprocCfg(
    target_size=512,
    sigma = 10.0,
    flag_refine_mask = False,
    open_ksize = 5,
    close_ksize = 5
)
pre = ImgPreproc(Imgcfg)

for l in labels:
    input_dir = input_dir_base / l
    for img_path in sorted(input_dir.glob("*.jpg")):
        img = cv.imread(str(img_path))
        if img is None:
            continue
        
        preOutput = pre.process(img)


        out_name_img_sq = f"{img_path.stem}_img_sq.png"
        out_img_sq_path = output_dir / out_name_img_sq
        cv.imwrite(str(out_img_sq_path), preOutput.img)

        out_name_mask_sq = f"{img_path.stem}_mask_sq.png"
        out_mask_sq_path = output_dir / out_name_mask_sq
        cv.imwrite(str(out_mask_sq_path), preOutput.mask)





In [8]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

import cv2 as cv
import numpy as np

run = 12
dataset = 4

# --- Configuración de paths ---
input_dir_base = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"
output_dir_base = PROJECT_ROOT / "Database" / "tmp" / f"imgPreprocTry{run:02d}"
output_dir_base.mkdir(parents=True, exist_ok=True)

labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

Imgcfg = ImgPreprocCfg(
    target_size=512,
    sigma = 10.0,
    flag_refine_mask = False,
    open_ksize = 5,
    close_ksize = 5
)

pre = ImgPreproc(Imgcfg)

# Pares (threshold1, threshold2) para probar
canny_params = [
    (30, 90),
    (50, 150),
    (70, 210),
]

# Suavizado previo (ajustable)
GAUSS_SIGMA = 1.55


for l in labels:
    input_dir = input_dir_base / l
    output_dir = output_dir_base / l
    output_dir.mkdir(parents=True, exist_ok=True)
    
    for img_path in sorted(input_dir.glob("*.jpg")):
        img_bgr = cv.imread(str(img_path))
        if img_bgr is None:
            continue

        # Escala de grises
        gray = cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)

        # Blur suave para reducir ruido de textura del papel
        gray_blur = cv.GaussianBlur(gray, (0, 0), sigmaX=GAUSS_SIGMA)

        for t1, t2 in canny_params:
            edges = cv.Canny(gray_blur, threshold1=t1, threshold2=t2)

            out_name = f"{img_path.stem}_canny_t{t1}_{t2}.png"
            out_path = output_dir / out_name
            cv.imwrite(str(out_path), edges)

In [10]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

import cv2 as cv
import numpy as np

from Code.image import ImgPreproc, ImgPreprocCfg

run = 13
dataset = 4

# --- Configuración de paths ---
input_dir_base = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"
output_dir_base = PROJECT_ROOT / "Database" / "tmp" / f"imgPreprocTry{run:02d}"
output_dir_base.mkdir(parents=True, exist_ok=True)

labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

Imgcfg = ImgPreprocCfg(
    target_size=512,
    sigma = 2.0,
    flag_refine_mask = False,
    open_ksize = 5,
    close_ksize = 5
)

pre = ImgPreproc(Imgcfg)

# Pares (threshold1, threshold2) para probar
canny_params = [
    (30, 90),
    (50, 150),
    (70, 210),
]

for l in labels:
    input_dir = input_dir_base / l
    output_dir = output_dir_base / l
    output_dir.mkdir(parents=True, exist_ok=True)
    
    for img_path in sorted(input_dir.glob("*.jpg")):
        img_bgr = cv.imread(str(img_path))
        if img_bgr is None:
            continue

        # --- Pipeline real de preproc ---
        out = pre.process(img_bgr, blacknwhite=True)

        gray_sq = (out.img * 255).astype(np.uint8)  # Canny quiere uint8
        mask_sq = out.mask

        # Opcional: trabajar solo dentro del objeto
        gray_obj = cv.bitwise_and(gray_sq, gray_sq, mask=mask_sq)

        for t1, t2 in canny_params:
            edges = cv.Canny(gray_obj, threshold1=t1, threshold2=t2)

            out_name = f"{img_path.stem}_canny_t{t1}_{t2}.png"
            out_path = output_dir / out_name
            cv.imwrite(str(out_path), edges)